In [58]:
import json
import pandas as pd
import requests
import time
from dotenv import load_dotenv

load_dotenv()

import sys
import os

# Get the parent directory
parent_dir = os.path.abspath('..')

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from azure.identity import DefaultAzureCredential
from conversation_store import ConversationStore

key = DefaultAzureCredential()
db = ConversationStore(
        url=os.getenv("COSMOSDB_ENDPOINT"),
        key=key,
        database_name=os.getenv("COSMOSDB_DATABASE_NAME"),
        container_name=os.getenv("COSMOSDB_CONTAINER_USER_NAME")
    )


In [59]:
#history = db.list_user_chats('rm9')
#print(history)


In [60]:
def get_agentic_response(row):
    import requests  # Import within the function for safety in multi-threaded environments
    
    query = row['Queries']
    
    # Prepare the payload for the API call
    payload = {
        "user_id": "rm9",
        "message": query
    }


    # Make the API call and handle exceptions
    try:
        response = requests.post('http://localhost:7071/api/http_trigger', json=payload)
        
        # Check if the response status is OK
        if response.status_code == 200:
            assistant_response = response.json()
            
            for message in reversed(assistant_response['reply']):
                if message.get('role') == 'assistant':
                    assistant_message = message
                    break
                
            if assistant_message:
                content = assistant_message.get('content', '')
                name = assistant_message.get('name', '')
                print(f"response: agent = {name} content = {content}")
        else:
            content = f"Error: Received status code {response.status_code}"
            name = ''
    except requests.exceptions.RequestException as e:
        content = f"Error: {e}"
        name = ''
    except ValueError:
        content = "Error: Unable to parse the response from the server."
        name = ''

     # Add a delay (e.g., 5 second) between requests
    time.sleep(5)

    # Return the content and name as a Series
    return pd.Series({'agentic_answer': content, 'agent_name': name})


In [61]:
df = pd.read_csv('/home/aga/azureai/aoai-insurance-rm-vanilla-agents/travel_question_bank_answered_best_small.csv')

# Display the DataFrame
#print(df)

# loop the 'Queries' field to test
#for query in df['Queries']:
#    print(query)


In [ ]:
# Apply the function to each row
df[['agentic_answer', 'agent_name']] = df.apply(get_agentic_response, axis=1)

In [63]:
df[['Queries', 'agentic_answer', 'agent_name']].head()

,Queries,agentic_answer,agent_name
0,"What countries are is travel area 1, 2 and 3?",The information about specific travel areas an...,Product
1,Please let me know the full claim procedure an...,Here is the procedure for filing a personal ac...,Product
2,What are the activities that are covered durin...,Zurich Travel Insurance covers various activit...,Product
